In [366]:
#import the libraries for machine learning competition
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_squared_error

plt.style.use('Solarize_Light2')
%matplotlib inline

In [367]:
#read train df
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.shape

(1400, 29)

In [368]:
#read wheather data
weather = pd.read_csv('weather.csv')
weather.describe()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,1978.000000,1884.000000,1915.000000,1824.00000,0.0,1397.000000,1663.000000,0.0,1018.000000,0.0
mean,18.806067,13.759926,25.855248,0.41080,NaN,147.866142,16.403548,NaN,1016.178487,NaN
std,6.949847,6.934710,8.182717,2.48288,NaN,95.944819,6.495765,NaN,6.963823,NaN
min,4.200000,-0.200000,6.800000,0.00000,NaN,1.000000,4.900000,NaN,943.600000,NaN
25%,12.525000,7.800000,18.800000,0.00000,NaN,72.000000,11.600000,NaN,1013.200000,NaN
50%,18.500000,13.800000,25.300000,0.00000,NaN,97.000000,15.300000,NaN,1015.750000,NaN
75%,25.000000,19.800000,31.800000,0.00000,NaN,242.000000,19.900000,NaN,1019.600000,NaN
max,38.300000,30.400000,46.500000,51.80000,NaN,359.000000,46.000000,NaN,1033.700000,NaN


In [369]:
#drop the columns with more than 50% missing values in the weather data
weather = weather.drop(["snow"], axis=1)


In [370]:
#show the columns which have more than 50% missing values in weather
weather.isnull().sum()[weather.isnull().sum()>0.5*len(weather)]

wpgt    1978
tsun    1978
dtype: int64

In [371]:
#drop wpgt and tsun columns
weather = weather.drop(["wpgt", "tsun"], axis=1)

In [372]:
weather.describe()

,tavg,tmin,tmax,prcp,wdir,wspd,pres
count,1978.000000,1884.000000,1915.000000,1824.00000,1397.000000,1663.000000,1018.000000
mean,18.806067,13.759926,25.855248,0.41080,147.866142,16.403548,1016.178487
std,6.949847,6.934710,8.182717,2.48288,95.944819,6.495765,6.963823
min,4.200000,-0.200000,6.800000,0.00000,1.000000,4.900000,943.600000
25%,12.525000,7.800000,18.800000,0.00000,72.000000,11.600000,1013.200000
50%,18.500000,13.800000,25.300000,0.00000,97.000000,15.300000,1015.750000
75%,25.000000,19.800000,31.800000,0.00000,242.000000,19.900000,1019.600000
max,38.300000,30.400000,46.500000,51.80000,359.000000,46.000000,1033.700000


In [373]:
#fill the missing values with the mean of the column
weather = weather.fillna(weather.mean())

C:\Users\user\AppData\Local\Temp\ipykernel_2820\2142933167.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  weather = weather.fillna(weather.mean())


In [374]:
#copy the train df to a new df
train_copy = train.copy()
test_copy = test.copy()

In [375]:
train_copy.head(2)

,date,MELLEGUE,BEN METIR,KASSEB,BARBARA,SIDI SALEM,BOU-HEURTMA,JOUMINE,GHEZALA,SEJNANE,...,SIDI AÏCH,EL BREK,BEZIRK,CHIBA,MASRI,LEBNA,HMA,ABID,Zarga,Ziatine
0,2014-01-08,26.721,48.091,67.485,60.787,448.591,71.317,84.411,7.883,89.022,...,0.201,2.4,3.255,0.293,2.822,9.82,1.604,4.608,NaN,NaN
1,2014-01-09,21.315,44.527,64.278,58.725,400.528,62.800,74.739,7.221,79.611,...,0.201,2.4,2.201,0.134,1.549,8.17,1.491,3.558,NaN,NaN


In [376]:
#change the date column to datetime type
train_copy['date'] = pd.to_datetime(train_copy['date'])

In [377]:
train_copy.head(2)

,date,MELLEGUE,BEN METIR,KASSEB,BARBARA,SIDI SALEM,BOU-HEURTMA,JOUMINE,GHEZALA,SEJNANE,...,SIDI AÏCH,EL BREK,BEZIRK,CHIBA,MASRI,LEBNA,HMA,ABID,Zarga,Ziatine
0,2014-01-08,26.721,48.091,67.485,60.787,448.591,71.317,84.411,7.883,89.022,...,0.201,2.4,3.255,0.293,2.822,9.82,1.604,4.608,NaN,NaN
1,2014-01-09,21.315,44.527,64.278,58.725,400.528,62.800,74.739,7.221,79.611,...,0.201,2.4,2.201,0.134,1.549,8.17,1.491,3.558,NaN,NaN


In [378]:
#iterate through the rows
for i in range(len(train_copy)-1):
    if (train_copy['date'][i+1]-train_copy['date'][i]).days>1:
        train_copy = train_copy.drop(i,axis=0)


In [379]:
#iterate through the column names
cols = train_copy.columns
for col in cols:
    #verify if the column isn't the day nor the year nor the month
    if col not in ['date']:
        #create a new column that contains the value for the current column in the next row
        train_copy[col + '_target'] = train_copy[col].shift(-1)
    #create a new column that contains the value for the current column in the next row


In [380]:
train_copy.columns

Index(['date', 'MELLEGUE', 'BEN METIR', 'KASSEB', 'BARBARA', 'SIDI SALEM',
       'BOU-HEURTMA', 'JOUMINE', 'GHEZALA', 'SEJNANE', 'S. EL BARRAK',
       'SILIANA', 'LAKHMESS', 'RMIL', 'BIR M'CHERGA', 'RMEL', 'NEBHANA',
       'SIDI SAAD', 'EL HAOUAREB', 'SIDI AÏCH', 'EL BREK', 'BEZIRK', 'CHIBA',
       'MASRI', 'LEBNA', 'HMA', 'ABID', 'Zarga', 'Ziatine', 'MELLEGUE_target',
       'BEN METIR_target', 'KASSEB_target', 'BARBARA_target',
       'SIDI SALEM_target', 'BOU-HEURTMA_target', 'JOUMINE_target',
       'GHEZALA_target', 'SEJNANE_target', 'S. EL BARRAK_target',
       'SILIANA_target', 'LAKHMESS_target', 'RMIL_target',
       'BIR M'CHERGA_target', 'RMEL_target', 'NEBHANA_target',
       'SIDI SAAD_target', 'EL HAOUAREB_target', 'SIDI AÏCH_target',
       'EL BREK_target', 'BEZIRK_target', 'CHIBA_target', 'MASRI_target',
       'LEBNA_target', 'HMA_target', 'ABID_target', 'Zarga_target',
       'Ziatine_target'],
      dtype='object')

In [381]:
weather.describe()

,tavg,tmin,tmax,prcp,wdir,wspd,pres
count,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000
mean,18.806067,13.759926,25.855248,0.410800,147.866142,16.403548,1016.178487
std,6.949847,6.767840,8.051285,2.384217,80.623362,5.955833,4.994648
min,4.200000,-0.200000,6.800000,0.000000,1.000000,4.900000,943.600000
25%,12.525000,8.225000,18.800000,0.000000,82.000000,12.300000,1015.600000
50%,18.500000,13.759926,25.800000,0.000000,147.866142,16.403548,1016.178487
75%,25.000000,19.800000,31.800000,0.000000,206.000000,18.900000,1016.178487
max,38.300000,30.400000,46.500000,51.800000,359.000000,46.000000,1033.700000


In [382]:
#make a copy of the weather df
weather_copy = weather.copy()

In [383]:
weather['date'] = pd.to_datetime(weather['date'])
train_copy = pd.merge(train_copy, weather, on='date', how='left')

In [384]:
train_copy.columns

Index(['date', 'MELLEGUE', 'BEN METIR', 'KASSEB', 'BARBARA', 'SIDI SALEM',
       'BOU-HEURTMA', 'JOUMINE', 'GHEZALA', 'SEJNANE', 'S. EL BARRAK',
       'SILIANA', 'LAKHMESS', 'RMIL', 'BIR M'CHERGA', 'RMEL', 'NEBHANA',
       'SIDI SAAD', 'EL HAOUAREB', 'SIDI AÏCH', 'EL BREK', 'BEZIRK', 'CHIBA',
       'MASRI', 'LEBNA', 'HMA', 'ABID', 'Zarga', 'Ziatine', 'MELLEGUE_target',
       'BEN METIR_target', 'KASSEB_target', 'BARBARA_target',
       'SIDI SALEM_target', 'BOU-HEURTMA_target', 'JOUMINE_target',
       'GHEZALA_target', 'SEJNANE_target', 'S. EL BARRAK_target',
       'SILIANA_target', 'LAKHMESS_target', 'RMIL_target',
       'BIR M'CHERGA_target', 'RMEL_target', 'NEBHANA_target',
       'SIDI SAAD_target', 'EL HAOUAREB_target', 'SIDI AÏCH_target',
       'EL BREK_target', 'BEZIRK_target', 'CHIBA_target', 'MASRI_target',
       'LEBNA_target', 'HMA_target', 'ABID_target', 'Zarga_target',
       'Ziatine_target', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd',
       'pres'],
      

In [385]:
train_copy.isnull().sum()[train_copy.isnull().sum()>0.5*len(train_copy)]

Zarga             679
Ziatine           679
Zarga_target      679
Ziatine_target    679
dtype: int64

In [386]:
train_copy[["MELLEGUE","MELLEGUE_target"]].head(5)

,MELLEGUE,MELLEGUE_target
0,26.721,21.315
1,21.315,26.519
2,26.519,21.230
3,21.230,26.282
4,26.282,22.446


In [387]:
#fill the missing values with the mean of the column in a new df
train_copy_mean = train_copy.fillna(train_copy.mean())

C:\Users\user\AppData\Local\Temp\ipykernel_2820\3350475074.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  train_copy_mean = train_copy.fillna(train_copy.mean())


In [388]:
train_copy_mean.tail(2)

,date,MELLEGUE,BEN METIR,KASSEB,BARBARA,SIDI SALEM,BOU-HEURTMA,JOUMINE,GHEZALA,SEJNANE,...,ABID_target,Zarga_target,Ziatine_target,tavg,tmin,tmax,prcp,wdir,wspd,pres
1242,2019-02-04,35.407,58.590,63.044,65.100,564.146,108.697,98.046,9.939,111.721,...,8.928000,23.986000,32.917000,7.6,3.8,10.8,0.4108,279.0,24.1,1014.5
1243,2019-02-05,36.898,55.998,61.972,64.462,573.260,109.365,97.399,9.798,106.741,...,3.894588,17.242253,24.270202,10.1,6.8,11.8,0.0000,311.0,28.3,1015.4


In [389]:
#read the sample submission file
sample_submission = pd.read_csv('SampleSubmission.csv')

In [390]:
sample_submission['date'] = pd.to_datetime(sample_submission['date'])
sample_submission = pd.merge(sample_submission, weather, on='date', how='left')

In [391]:
#select the columns having in the column name the word target
cols = [col for col in train_copy_mean.columns if 'target' in col]
#split the train data into train and test
X_train, X_test, y_train, y_test = train_test_split(train_copy_mean.drop(cols, axis=1), train_copy_mean[cols], test_size=0.2, random_state=42)


In [392]:
#drop the date column from the train and test data
X_train = X_train.drop(['date'], axis=1)
X_test = X_test.drop(['date'], axis=1)

In [393]:
#create the multioutput regressor
reg = MultiOutputRegressor(ensemble.GradientBoostingRegressor())
reg.fit(X_train, y_train)
Y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, Y_pred, squared=False)
mse


6.08755355162946

In [394]:
#get the last row
last_row = train_copy_mean.tail(1)
#drop the date column
last_row = last_row.drop(['date'], axis=1)
#predict the values for the last row
last_row = last_row.drop(cols, axis=1)
pred = reg.predict(last_row)
sample_submission.iloc[0,1:29] = pred[0]

In [395]:
row = sample_submission.iloc[0,:]
row = row.drop(['date'])
row = row.to_frame()
row


,0
MELLEGUE,36.713717
BEN METIR,56.055205
KASSEB,62.743448
BARBARA,59.753729
SIDI SALEM,583.440982
BOU-HEURTMA,103.721235
JOUMINE,96.355834
GHEZALA,8.857292
SEJNANE,103.252571
S. EL BARRAK,279.475268


In [396]:
#replace the missing values with mean
sample_submission = sample_submission.fillna(sample_submission.mean())

C:\Users\user\AppData\Local\Temp\ipykernel_2820\1698372725.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  sample_submission = sample_submission.fillna(sample_submission.mean())


In [397]:
#insert the predicted row in the first row of the sample submission df
for i in range(len(sample_submission)-1):
    row = sample_submission.iloc[i,:]
    row = row.drop(['date'])
    row = row.to_frame()
    pred1 = reg.predict(row.T)
    sample_submission.iloc[i+1,1:29] = pred1[0]


In [398]:
# #import hist_gradient_boosting_regressor
# from sklearn.ensemble import HistGradientBoostingRegressor
# model = HistGradientBoostingRegressor()

# cols = ['MELLEGUE', 'BEN METIR', 'KASSEB', 'BARBARA', 'SIDI SALEM', 'BOU-HEURTMA', 'JOUMINE', 'GHEZALA', 'SEJNANE', 'S. EL BARRAK',
#        'SILIANA', 'LAKHMESS', 'RMIL', "BIR M'CHERGA", 'RMEL', 'NEBHANA',
#        'SIDI SAAD', 'EL HAOUAREB', 'SIDI AÏCH', 'EL BREK', 'BEZIRK', 'CHIBA',
#        'MASRI', 'LEBNA', 'HMA', 'ABID', 'Zarga', 'Ziatine']
# weather_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'pres']
# #iterate through the list
# for col in cols:
#     predictable_cols = [col] + weather_cols
#     #split the train_copy_mean df into train and test
#     X_train, X_test, y_train, y_test = train_test_split(train_copy_mean[predictable_cols], train_copy_mean[col+"_target"], test_size=0.2, random_state=42)
#     model.fit(X_train, y_train)
#     #get the last row of the train_copy_mean df
#     last_row = train_copy_mean.tail(1)
#     #drop the date and the target columns
#     last_row = last_row.drop(["date",col+"_target"], axis=1)
#     #reformulate the last row to be predictable
#     last_row = last_row[predictable_cols]
#     #predict the value of the target column
#     pred = model.predict(last_row)
#     #insert the predicted value in the sample_submission df in the firsrt row
#     sample_submission.loc[0,col] = pred[0]
#     for i in range(len(sample_submission)-1):
#         #create a row that contains the date of the next day, the weather data of the next day and the value of the current col
#         row = pd.DataFrame([[sample_submission[col][i],sample_submission["tavg"][i+1],sample_submission["tmin"][i+1],sample_submission["tmax"][i+1],sample_submission["prcp"][i+1],sample_submission["wdir"][i+1],sample_submission["wspd"][i+1],sample_submission["pres"][i+1]]], columns=[col,"tavg","tmin","tmax","prcp","wdir","wspd","pres"])
#         #predict the value of the target column
#         pred = model.predict(row)
#         #insert the predicted value in the sample_submission df in the next row
#         sample_submission.loc[i+1,col] = pred[0]

In [399]:
sample_submission.columns


Index(['date', 'MELLEGUE', 'BEN METIR', 'KASSEB', 'BARBARA', 'SIDI SALEM',
       'BOU-HEURTMA', 'JOUMINE', 'GHEZALA', 'SEJNANE', 'S. EL BARRAK',
       'SILIANA', 'LAKHMESS', 'RMIL', 'BIR M'CHERGA', 'RMEL', 'NEBHANA',
       'SIDI SAAD', 'EL HAOUAREB', 'SIDI AÏCH', 'EL BREK', 'BEZIRK', 'CHIBA',
       'MASRI', 'LEBNA', 'HMA', 'ABID', 'Zarga', 'Ziatine', 'tavg', 'tmin',
       'tmax', 'prcp', 'wdir', 'wspd', 'pres'],
      dtype='object')

In [400]:
weather_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'pres']
#drop weather cols
sample_submission = sample_submission.drop(weather_cols, axis=1)
sample_submission.to_csv("submission.csv", index=False)

